In [1]:
import os
import numpy as np
import h5py
import dldata.metrics.utils as utils
import dldata.stimulus_sets.hvm as hvm
from sklearn.model_selection import KFold
from sklearn.preprocessing import scale
from scipy.stats import pearsonr
from cubemap.mapper import Mapper
import tensorflow as tf

np.random.seed([0])
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)

dataset_hvm = hvm.HvMWithDiscfade()
meta_hvm = dataset_hvm.meta
neurons = dataset_hvm.neuronal_features
v4_data = neurons[:, dataset_hvm.V4_NEURONS]

/braintree/home/bashivan/anaconda2/envs/dldata/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/braintree/home/bashivan/anaconda2/envs/dldata/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/braintree/home/bashivan/anaconda2/envs/dldata/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection

Can't import separate mcc package
Can't import asgd.
Can't import scikits stuff
common attributes, forcing a renaming ...
Using default renamer ...
('Replacing columns', ['rxz', 'rxy', 'ryz'])


# Separable (Mask-Mix) Mapping

In [3]:
kf = KFold(n_splits=2)


with h5py.File('/braintree/data2/active/users/bashivan/model_features/alexnet_retina299_alexpreprocess.h5') as features_h5_hvm:
    X = np.array(features_h5_hvm['mdl_conv3'])

Y = scale(v4_data)
    
num_images = X.shape[0]
num_neurons = Y.shape[-1]

# mapping accuracy is very sensitive to the choice of regularization parameters ls and ld
ls = 1.  # regularization parameter for the spatial mask weights
ld = 0.1  # regularization parameter for the mixing weights
 
print('Mapping with ls={}, ld={}'.format(ls, ld))

preds = np.zeros_like(Y)
for i, (train_ind, test_ind) in enumerate(kf.split(X)):
    mapper = Mapper(graph=None, max_epochs=30, tol=0.1, 
                    init_lr=0.1, batch_size=10, ls=ls, ld=ld, num_neurons=num_neurons,
                    map_type='separable', inits=None, decay_rate=10, log_rate=10)

    mapper.fit(X[train_ind], Y[train_ind])
    preds[test_ind] = mapper.predict(X[test_ind])
scores = np.array([pearsonr(preds[:, i], Y[:, i])[0] for i in range(preds.shape[-1])])
print('Scores: ', np.median(scores))


/braintree/home/bashivan/anaconda2/envs/dldata/lib/python2.7/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/braintree/home/bashivan/anaconda2/envs/dldata/lib/python2.7/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


Mapping with ls=0.1, ld=0.1
Initializing...
Epoch: 1, Err Loss: 7730.69, Reg Loss: 7331.64
Epoch: 11, Err Loss: 110.75, Reg Loss: 308.84
Epoch: 21, Err Loss: 45.45, Reg Loss: 27.77
Epoch: 31, Err Loss: 44.32, Reg Loss: 12.13
Epoch: 41, Err Loss: 41.48, Reg Loss: 10.24
Epoch: 51, Err Loss: 43.03, Reg Loss: 8.33
Epoch: 61, Err Loss: 39.60, Reg Loss: 8.11
Epoch: 71, Err Loss: 43.45, Reg Loss: 7.86
Initializing...
Epoch: 1, Err Loss: 7673.11, Reg Loss: 7456.63
Epoch: 11, Err Loss: 135.07, Reg Loss: 361.70
Epoch: 21, Err Loss: 55.32, Reg Loss: 34.77
Epoch: 31, Err Loss: 54.12, Reg Loss: 16.47
Epoch: 41, Err Loss: 49.14, Reg Loss: 14.29
Epoch: 51, Err Loss: 49.66, Reg Loss: 12.04
Epoch: 61, Err Loss: 51.16, Reg Loss: 11.78
Epoch: 71, Err Loss: 52.17, Reg Loss: 11.47
('Scores: ', 0.5975703)


In [4]:
# PCR Comparison
from sklearn import linear_model
from sklearn.decomposition import PCA

Y = scale(v4_data)

features = X.reshape(X.shape[0], -1)
pca = PCA(n_components=1000, whiten=False)
model_pcs = pca.fit_transform(features)

preds = np.zeros_like(Y)
for i, (train_ind, test_ind) in enumerate(kf.split(model_pcs)):
    print('working on fold {0}/{1}'.format(i+1, 2))
    clf = linear_model.Ridge(alpha=10)
    clf.fit(model_pcs[train_ind], Y[train_ind])

    # print the test performance 
    preds[test_ind] = clf.predict(model_pcs[test_ind])
scores = np.array([pearsonr(preds[:, i], Y[:, i])[0] for i in range(preds.shape[-1])])
print('Scores: ', np.median(scores))


working on fold 1/2
working on fold 2/2
('Scores: ', 0.5128074)
